In [ ]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp
import matplotlib.pyplot as plt
import seaborn as sns

import implicit
import lightfm
import warnings
from validation_prepare import *
warnings.filterwarnings('ignore')
%matplotlib inline

C:\Users\yabom\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [ ]:
def get_recomend(model,X,fe,test_users,user_history_films,user_features = None,item_features = None,mode = 'lightfm',N = 20,is_filter_history = True):
    # Сделаем какой-нибудь сабмит из lightfm
    result = {}
    #'implicit'#'lightfm'
    cnt = 0
    for user_uid in tqdm.tqdm(test_users):

        # transform user_uid to model's internal user category
        try:
            user_cat = fe.match_user_row[user_uid]
        except LookupError:
            continue
        
        if mode == 'lightfm':
            if is_filter_history:
                need_cols = np.array(list(set(np.arange(X.shape[1])) - user_history_films.get(user_cat, set())))
            else:
                need_cols = np.array(list(set(np.arange(X.shape[1]))))
            recs = model.predict(user_cat,need_cols,item_features=item_features,user_features=user_features)
            need_movies = np.argsort(-recs)[:N]
            result[user_uid] = [int(fe.train_movie_match_row_movie[i]) for i in need_movies]
            
                
        # переводим в фильмы

        # perform inference
        if mode == 'implicit':
            if cnt == 0:
                ratings_matrix_T = X.tocsr()
            if is_filter_history:
                recs = model.recommend(
                    user_cat,
                    X.tocsr(),
                    N=N,
                    filter_already_liked_items=True,
                    filter_items=user_history_films.get(user_uid, set())
                )
            else:
                recs = model.recommend(
                    user_cat,
                    X.tocsr(),
                    N=N,filter_already_liked_items=False,
                )
            result[user_uid] = [int(fe.train_movie_match_row_movie[i]) for i, _ in recs]
            # drop scores and transform model's internal elelemnt category to element_uid for every prediction
        # also convert np.uint64 to int so it could be json serialized later
        cnt += 1
    return result

def get_predict (model,X,fe,test_users,user_history_films,user_features = None,item_features = None,mode = 'lightfm',):
    # Сделаем какой-нибудь сабмит из lightfm
    result = {}
    #'implicit'#'lightfm'
    cnt = 0
    for user_uid in tqdm.tqdm(test_users):

        # transform user_uid to model's internal user category
        try:
            user_cat = fe.match_user_row[user_uid]
        except LookupError:
            continue
        
        if mode == 'lightfm':
#             if is_filter_history:
#                 need_cols = np.array(list(set(np.arange(X.shape[1])) - user_history_films.get(user_cat, set())))
#             else:
            need_cols = np.array(list(set(np.arange(X.shape[1]))))
            recs = model.predict(user_cat,need_cols,item_features=item_features,user_features=user_features)
            #need_movies = np.argsort(-recs)[:N]
            result[user_uid] = recs#[int(fe.train_movie_match_row_movie[i]) for i in need_movies]
            
                
        # переводим в фильмы

        # perform inference
        if mode == 'implicit':
            if cnt == 0:
                ratings_matrix_T = X.tocsr()
            
            recs = model.rank_items(user_cat, X.tocsr(), np.arange(X.shape[1]), )
            recs = sorted(recs,key = lambda x:x[0])
            
            result[user_uid] = [score for i, score in recs]
            # drop scores and transform model's internal elelemnt category to element_uid for every prediction
        # also convert np.uint64 to int so it could be json serialized later
            cnt += 1
    return result

In [ ]:
pd.set_option('display.max_columns',100)

DATA_PATH = '../okko/orig_data'
PREPARED_PATH = './prepared_data/'

In [ ]:
actions = pd.read_pickle(PREPARED_PATH+'actions_one_table.pkl')

actions.sort_index(inplace = True) # На всякий случай, иначе деление не будет работать

actions.head()

In [ ]:
_,_,watch_actions,_ = get_target(actions)

actions = actions.join(watch_actions['rel_dur'])

actions['rel_dur'] = actions['rel_dur'].fillna(0).replace(np.inf,1)

In [ ]:
idx = get_train_test(actions)

In [ ]:
actions.iloc[idx[0]].index.get_level_values(2).max()

In [ ]:
actions.iloc[idx[1]].index.get_level_values(2).min()

In [ ]:
actions.iloc[idx[1]].index.get_level_values(2).max()

In [ ]:
actions.iloc[idx[2]].index.get_level_values(2).min()

In [ ]:
actions.consumption_mode.value_counts()

In [ ]:
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])


In [ ]:
actions.head()

In [ ]:
# Вроде не пересекается.
train,test,valid = actions.iloc[idx[0]],actions.iloc[idx[1]],actions.iloc[idx[2]]

In [ ]:
%time 
dur_being_train,dur_films_train,watch_actions_train,target_train = get_target(train)
dur_being_test,dur_films_test,watch_actions_test,target_test = get_target(test)
dur_being_valid,dur_films_valid,watch_actions_valid,target_valid = get_target(valid)

In [ ]:
# Получили фичи для фильмов
import pickle
with open(PREPARED_PATH+'catalogue_features.pkl','rb') as f:
    match_element_row,match_row_element,match_columns,element_matrix = pickle.load(f)
movie_match_columns = {i:ii for ii,i in enumerate(match_columns)}
movie_columns_match = {ii:i for ii,i in enumerate(match_columns)}

with open(PREPARED_PATH+'bag_of_attr_movie.pkl','rb') as f:
    bag_of_attr = pickle.load(f)

In [ ]:
fe = FeatureExtractor({'movie_attr_matrix':element_matrix,'movie_match_columns':movie_match_columns,
                       'movie_columns_match':movie_columns_match,'movie_match_row_movie':match_row_element,
                      'movie_match_movie_row':match_element_row,},bag_of_attr,is_censor = True,delimiter=-1,mode = 'duration',
                     target_col_name='rel_dur')

fe = FeatureExtractor({'movie_attr_matrix':element_matrix,'movie_match_columns':movie_match_columns,
                       'movie_columns_match':movie_columns_match,'movie_match_row_movie':match_row_element,
                      'movie_match_movie_row':match_element_row,},bag_of_attr,is_censor =False,delimiter=4,)

fe.fit(train)

In [ ]:
train_ = fe.transform(train)
test_ = fe.transform(test)
valid_ = fe.transform(valid)

train_.shape,test_.shape 

cfe = ColdFeatureExtractor(fe)

cfe.fit(train)

max(cfe.train_movie_cols)

# sp.csc_matrix(cfe.fitted_FE.movie_attr_matrix)[cfe.train_movie_rows,cfe.train_movie_cols]



train_res = cfe.transform(test)


print(train_res['train_user'].shape,train_res['test_user'].shape,train_res['new_test_user'].shape)

print(train_res['train_movie'].shape,train_res['test_movie'].shape)

In [ ]:
np.min(train_)

In [ ]:
'''
Здесь начинается русское поле экспериментов над параметрами моделей.
'''

In [ ]:
train_.shape,test_.shape,valid_.shape

In [ ]:
train_res['train_movie']

In [ ]:
%%time
from collections import defaultdict
a,b = train_.nonzero()
D_row_col_train = defaultdict(set)
D_user_movie_train = defaultdict(set)
for i,ii in tqdm.tqdm(zip(a,b),total = len(a)):
    D_row_col_train[i].add(ii)
    D_user_movie_train[fe.match_row_user[i]].add(fe.train_movie_match_row_movie[ii])

from collections import defaultdict
a,b = test_.nonzero()
D_row_col_test = defaultdict(set)
D_user_movie_test = defaultdict(set)
for i,ii in tqdm.tqdm(zip(a,b),total = len(a)):
    D_row_col_test[i].add(ii)
    D_user_movie_test[fe.match_row_user[i]].add(fe.train_movie_match_row_movie[ii])

from collections import defaultdict
a,b = valid_.nonzero()
D_row_col_valid = defaultdict(set)
D_user_movie_valid = defaultdict(set)
for i,ii in tqdm.tqdm(zip(a,b),total = len(a)):
    D_row_col_valid[i].add(ii)
    D_user_movie_valid[fe.match_row_user[i]].add(fe.train_movie_match_row_movie[ii])

In [ ]:

seed = 0
epochs = 30
num_threads=4

model = lightfm.LightFM(loss = 'warp',random_state=seed)

In [ ]:
from implicit.nearest_neighbours import TFIDFRecommender,CosineRecommender,NearestNeighboursScorer,BM25Recommender,bm25_weight
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import mean_average_precision_at_k,precision_at_k

In [ ]:
from lightfm.evaluation import precision_at_k,auc_score

In [ ]:
np.max(train_)

In [ ]:
model_im = BM25Recommender()#CosineRecommender()
model_im.fit(train_.T)

In [ ]:
# recs_model_im_test = get_predict(model_im,train_,fe,D_user_movie_test.keys(),D_row_col_train,mode = 'implicit')

# list(recs_model_im_test.keys())[0]

# len(recs_model_im_test[17])

In [ ]:
model = lightfm.LightFM(loss = 'logistic',random_state=seed,max_sampled=100,no_components=100)
model.fit(train_,epochs=30,)

In [ ]:
# Здесь простецкий стекинг делать

In [ ]:
recs_model_test = get_predict(model,train_,fe,D_user_movie_test.keys(),D_row_col_train,mode = 'lightfm')
len(recs_model_test[list(recs_model_test.keys())[0]])
len(recs_model_test[17])

In [ ]:
recs_model_valid = get_predict(model,train_,fe,D_user_movie_valid.keys(),D_row_col_train,mode = 'lightfm')
len(recs_model_valid[list(recs_model_valid.keys())[0]])
#len(recs_model_valid[17])

In [ ]:
recs_model_test_true = get_predict(model,train_,fe,test_users,D_row_col_train,mode = 'lightfm')
len(recs_model_test_true[list(recs_model_test_true.keys())[0]]),len(recs_model_test_true)

In [ ]:
with open(os.path.join(DATA_PATH, 'catalogue.json'), 'r') as f:
    catalogue = json.load(f)
    
catalogue = {int(k): v for k, v in catalogue.items()}

In [ ]:
s_attr = set()
for i,ii in catalogue.items():
    s_attr.update(set(ii['attributes']))

In [ ]:
s_attr = list(s_attr)
s_attr = sorted(s_attr)

In [ ]:
s_dict_attr = {i:ii for ii,i in enumerate(s_attr)}

In [ ]:
def get_movie_feat(movie_id,data_catalogue_orig,shift = 0):
    l = catalogue[movie_id]['attributes']
    return {(s_dict_attr[i]+shift):1 for i in l}
def convert_to_range(X,data_catalogue_orig,path = PREPARED_PATH+'train.txt', label_name = 'target',need_feat = None):
    '''
    конвертируем в libsvm вместе с query файлом
    '''
    # Сначала рейтинг 0...10
    buf_tr = []
    buf_query = []
    cnt = 0
    for i in tqdm.tqdm(np.unique(X.index.get_level_values(0))):
        cnt_film = 0
        temp = X.loc[i]
        for movie_id in temp.index.get_level_values(0):
            row = temp.loc[movie_id,need_feat].to_dict()
            row = {need_feat.index(i):ii for i,ii in row.items()}
            dict_row = get_movie_feat(movie_id,data_catalogue_orig,shift = len(row))
            dict_row.update(row)
            s_movie = str(int(temp.loc[movie_id,label_name]))+' '
            dict_row = dict(sorted(dict_row.items()))
            cnt+=1
            for k,kk in dict_row.items():
                s_movie+= str(k)+':'+str(kk)+' '
            s_movie+='\n'
            buf_tr.append(s_movie)
            cnt_film+=1
            cnt+=1
        buf_query.append(str(cnt_film)+'\n')
    with open(path,'w') as f:
        for i in buf_tr:
            f.write(i)
    with open(path+'.query','w') as f:
        for i in buf_query:
            f.write(i)

In [ ]:
wm = pd.read_pickle(PREPARED_PATH+'some_wm.pkl')
wm_cut = wm.loc[~wm.rating.isnull()]

In [ ]:
wm_cut.head()

In [ ]:
# !help('modules')

In [ ]:
# from importlib import reload
# reload('validation_prepare.py')

In [ ]:
idx_wm = get_train_test(wm,mode = 'by_time_wm')

In [ ]:
train,test,valid = wm.iloc[idx_wm[0]],wm.iloc[idx_wm[1]],wm.iloc[idx_wm[2]]

In [ ]:
train.first_ts.max(),test.first_ts.min()

In [ ]:
# Пока оставим только тру рейтинги
train,test,valid = train.dropna(subset = ['rating']),test.dropna(subset = ['rating']),valid.dropna(subset = ['rating'])

In [ ]:
need_feat =['rel_dur','diff_ts','duration','feature_1','feature_2','feature_3','feature_4','feature_5','num_watched_user'
            ,'probably_kp_rate']

In [ ]:
import os

In [ ]:
# if not os.path.exists(PREPARED_PATH+'train.txt'):
convert_to_range(train,catalogue,label_name = 'rating',need_feat = need_feat)

In [ ]:
#if not os.path.exists(PREPARED_PATH+'test.txt'):
convert_to_range(test,catalogue,label_name = 'rating',need_feat = need_feat,path = PREPARED_PATH+'test.txt')

In [ ]:
from sklearn.datasets import load_svmlight_file

In [ ]:
import lightgbm

In [ ]:
x,y = load_svmlight_file(PREPARED_PATH+'test.txt')

In [ ]:
with open(PREPARED_PATH+'train.txt.query') as f:
    q_train = []
    for i in f:
        q_train.append(int(i.replace('\n','')))
with open(PREPARED_PATH+'test.txt.query') as f:
    q_test = []
    for i in f:
        q_test.append(int(i.replace('\n','')))

In [ ]:
lgb_train = lightgbm.Dataset(PREPARED_PATH+'train.txt',group = q_train)
lgb_test = lightgbm.Dataset(PREPARED_PATH+'test.txt',group = q_test)

In [ ]:
need_feat[9]

In [ ]:
params = {
    'objective':'lambdarank',
#     'data': PREPARED_PATH+'train.txt',
#     'valid':PREPARED_PATH+'test.txt',
#     'metric':'map',
}
model = lightgbm.train(params,lgb_train,valid_sets=lgb_test,verbose_eval=True,num_boost_round=1000,early_stopping_rounds=100)

In [ ]:
p = model.predict(x)
p#.reshape((-1,11)).shape

In [ ]:
fig,ax = plt.subplots(figsize = (10,20))
lightgbm.plot_importance(model,ax = ax)

In [ ]:
res = get_recomend(model_im,train_,fe,D_user_movie_train.keys(),D_row_col_train,mode = 'implicit',is_filter_history=False)

metric(D_user_movie_train,res)

In [ ]:
res = get_recomend(model_im,train_,fe,D_user_movie_test.keys(),D_row_col_train,mode = 'implicit')

metric(D_user_movie_test,res)

In [ ]:
res = get_recomend(model_im,train_,fe,D_user_movie_valid.keys(),D_row_col_train,mode = 'implicit')

metric(D_user_movie_valid,res)

In [ ]:
#precision_at_k(model,train_,test_,k = 20)

In [ ]:
# Организуем hyperopt
import colorama
import hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials


space ={
        'loss': hp.choice( 'loss', ['logistic', 'warp' ] ),
        'learning_schedule': hp.choice("learning_schedule", ['adagrad','adadelta']),
        'rho':  hp.quniform('rho', 0.75, 0.99,0.05),
        'max_sampled': hp.quniform('max_sampled', 10, 1000,20),
        'learning_rate': hp.loguniform('learning_rate', -6.9, -1),  
        'no_components':hp.quniform('no_components',5,200,5),
       }
cur_best_loss = np.inf
cnt = 1


def get_params(space):
    px = dict()
    px['loss'] = space['loss']
    px['learning_schedule'] = space['learning_schedule']
    px['rho'] = min(0.99,space['rho'])
    px['max_sampled'] = int(space['max_sampled'])
    px['learning_rate'] = space['learning_rate']
    px['no_components'] = int(space['no_components'])
    
    return px
def objective(space):
    global cur_best_loss,cnt
    params = get_params(space)
    model = lightfm.LightFM(random_state=seed,**params,)
    model.fit(train_,epochs=30,)
    res = get_recomend(model,train_,fe,D_user_movie_test.keys(),D_row_col_train)
    #score = custom_mae(l.values,p.values)
    score = -metric(D_user_movie_test,res)
    if cnt%10 == 0:
        print('Попытка номер:',cnt)
    if score < cur_best_loss:
        cur_best_loss = score
        print(colorama.Fore.GREEN + 'NEW BEST LOSS={}'.format(-cur_best_loss) + colorama.Fore.RESET)
        print(params)
    cnt+=1
    return {'loss':score, 'status': STATUS_OK }
trials = Trials()
# best = hyperopt.fmin(fn=objective,
#                      space=space,
#                      algo=tpe.suggest,
#                      max_evals=200,
#                      trials=trials,
#                      verbose=2)
p={'loss': 'warp', 'learning_schedule': 'adagrad', 'rho': 0.75, 'max_sampled': 460, 'learning_rate': 0.005108939164793534, 'no_components': 160}

In [ ]:
res = get_recomend(model,train_,fe,D_user_movie_test.keys(),D_row_col_train)#test_users#,mode = 'implicit'


In [ ]:
metric(D_user_movie_test,res)

In [ ]:
# Валидация
res = get_recomend(model,valid_,fe,D_user_movie_valid.keys(),D_row_col_train)#,mode = 'implicit'
metric(D_user_movie_valid,res)

In [ ]:
len(D_user_movie_test)

In [ ]:
len(set(D_user_movie_test.keys()) & set(res.keys()))

In [ ]:
'''
Здесь заканчиваетс/ русское поле экспериментов
'''

In [ ]:
actions.head()

In [ ]:
wm = pd.read_pickle(PREPARED_PATH+'some_wm.pkl')

In [ ]:
wm.head()

In [ ]:
#Проверим, что не загибается на полном датасете

fe = FeatureExtractor({'movie_attr_matrix':element_matrix,'movie_match_columns':movie_match_columns,
                       'movie_columns_match':movie_columns_match,'movie_match_row_movie':match_row_element,
                      'movie_match_movie_row':match_element_row,},bag_of_attr,is_censor = True,delimiter=4,)


# fe = FeatureExtractor({'movie_attr_matrix':element_matrix,'movie_match_columns':movie_match_columns,
#                        'movie_columns_match':movie_columns_match,'movie_match_row_movie':match_row_element,
#                       'movie_match_movie_row':match_element_row,},bag_of_attr,is_censor = False)
X = fe.fit_transform(actions)

In [ ]:
cfe = ColdFeatureExtractor(fe)

res_action = cfe.fit_transform(actions)

In [ ]:
#res_action.keys()

In [ ]:
#len(fe.train_movie_match_movie_row),len(fe.match_user_row)

In [ ]:
X_user = res_action['train_user']
X_movie = res_action['train_movie']
X_user.shape,X_movie.shape

In [ ]:
# def fit_lightfm(train,item_features=None,seed = 0)
seed = 0
epochs = 30
num_threads=4

model = lightfm.LightFM(loss = 'warp',random_state=seed)


# model.fit(train_matrix,user_features = train_user,item_features = element_matrix,epochs = epochs,num_threads = num_threads,
#          verbose = True)


In [ ]:


%time model.fit(X,epochs = epochs,num_threads = num_threads,verbose = True)

In [ ]:
%%time
from collections import defaultdict
a,b = X.nonzero()
D = defaultdict(set)
for i,ii in tqdm.tqdm(zip(a,b),total = len(a)):
    D[i].add(ii)
# D = pd.DataFrame([a,b],index = ['row','columns']).T
# D = D.groupby('row').agg(lambda x:frozenset(x.values))
# D

In [ ]:
#D[1]

In [ ]:
len(fe.train_movie_match_row_movie)

In [ ]:
print('sparsity',X.nnz/(X.shape[0]*X.shape[1]))

In [ ]:
len( set(test_users) - set(fe.match_user_row.keys()))

In [ ]:
from implicit.nearest_neighbours import TFIDFRecommender,CosineRecommender,NearestNeighboursScorer



In [ ]:
# Для implicit  надо фильмы*юзеры матрицу.
# model = CosineRecommender()
# model.fit(X.T.tocsr())

In [ ]:
model = CosineRecommender()
model.fit(X.T)

In [ ]:
res = get_recomend(model,X,fe,test_users,D,mode = 'implicit')

In [ ]:
# Сделаем какой-нибудь сабмит из lightfm
result = {}
mode = 'lightfm'#'implicit'#'lightfm'
cnt = 0
for user_uid in tqdm.tqdm(test_users):
    
    # transform user_uid to model's internal user category
    try:
        user_cat = fe.match_user_row[user_uid]
    except LookupError:
        continue
    if mode == 'lightfm':
        need_cols = np.array(list(set(np.arange(X.shape[1])) - D.get(user_cat, set())))
        recs = model.predict(user_cat,need_cols,num_threads=num_threads)
        need_movies = np.argsort(-recs)[:20]
        result[user_uid] = [int(fe.train_movie_match_row_movie[i]) for i in need_movies]
    # переводим в фильмы
    
    # perform inference
    if mode == 'implicit':
        if cnt == 0:
            ratings_matrix_T = X.tocsr()
        recs = model.recommend(
            user_cat,
            X.tocsr(),
            N=20,
            filter_already_liked_items=True,
            filter_items=D.get(user_uid, set())
        )
        result[user_uid] = [int(fe.train_movie_match_row_movie[i]) for i, _ in recs]
        # drop scores and transform model's internal elelemnt category to element_uid for every prediction
    # also convert np.uint64 to int so it could be json serialized later
    cnt += 1

In [ ]:
user_cat

In [ ]:
len(fe.train_movie_match_row_movie)

In [ ]:
recs

In [ ]:
#len(need_cols),len(recs),type(recs)

In [ ]:
#plt.hist(recs)

In [ ]:
len(result)

In [ ]:

with open('answer.json', 'w') as f:
    json.dump(result, f)

In [ ]:
result[0]

In [ ]:
from lightfm.evaluation import auc_score,precision_at_k

# Compute and print the AUC score
train_auc = precision_at_k(model, te ,tr, k = 20,num_threads=4).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:
new_old_tr_te_2.keys()

In [ ]:
tr = new_old_tr_te_2['train_interactions']
te = new_old_tr_te_2['test_interactions']
uf = new_old_tr_te_2['user_features_train']
tf = new_old_tr_te_2['user_features_test']
print(tr.shape,te.shape,uf.shape,tf.shape)
# tr[(tr < 3) &  (tr>0)] = -1
# tr[(tr > 3)] = 1
# te[(tr < 3) &  (te>0)] = -1
# te[(tr > 3)] = 1

model.fit(train_matrix,user_features=uf,epochs = epochs,num_threads =num_threads,
         verbose = True)

In [ ]:
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt


In [ ]:
p = model.predict(0,np.arange(tr.shape[1]),user_features=tf[0,:],num_threads=num_threads)

In [ ]:
plt.hist(p)

In [ ]:
test_user[0,:]

In [ ]:
a = model.predict(0,np.arange(train_matrix.shape[1]),user_features=test_user[0,:],num_threads=num_threads)
len(a)

In [ ]:
from lightfm.evaluation import auc_score,precision_at_k

# Compute and print the AUC score
train_auc = precision_at_k(model, train_matrix,user_features = train_user,item_features = element_matrix ,num_threads=num_threads).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:
model.user_feature_map

In [ ]:
def get_answer(test_users_dict,test):
    pass